In [46]:
import os
import re
import pandas as pd
import json
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("/Users/anand/Documents/sayansi/workspaces/condaws/xlsx-json/custom.css", "r").read()
    HTML(''.format(styles))
    #return HTML(styles)
css_styling()

HTML("""
<style>
.renderjson a              { text-decoration: none !important; }
.renderjson .disclosure    { color: crimson;
                             font-size: 100%; }
.renderjson .syntax        { color: grey; }
.renderjson .string        { color: red; }
.renderjson .number        { color: cyan; }
.renderjson .boolean       { color: plum; }
.renderjson .key           { color: lightblue; }
.renderjson .keyword       { color: lightgoldenrodyellow; }
.renderjson .object.syntax { color: lightseagreen; }
.renderjson .array.syntax  { color: lightsalmon; }
/* unvisited link */

.renderjson a:link {
  color: blue;
}

/* visited link */
.renderjson a:visited {
  color: green;
}

/* mouse over link */
.renderjson a:hover {
  color: hotpink;
}

/* selected link */
.renderjson a:active {
  color: blue;
}
</style>
""")



In [47]:
class LoadExcel:
    
    fileName = None
    columnNames = None
    columnNameRowIndex = 0
    filePath = None
    qualifieldFilePath = None
    sheetNames = None
    
    def __init__(self,filePath=None, fileName = None,columnNames = None , sheetNames=None , columnNameRowIndex = 0):
        self.filePath = filePath
        self.fileName = fileName
        self.columnNames = columnNames
        self.columnNameRowIndex = columnNameRowIndex
        self.sheetNames = sheetNames
        if fileName is not None and filePath is not None:
            self.qualifieldFilePath = str(filePath) +"/"+ str(fileName)
            
    def loadFile(self, readsheet):
        fload = None        
        #print("Readin sgeet :"+readsheet)
        if isinstance(readsheet, str) :
            fload = pd.read_excel(self.qualifieldFilePath,sheet_name= readsheet)
        return fload
    
    def get_col_name_frm_row(self, df,rowno, startColNo,endColNo):
        a1 = df.iloc[rowno,startColNo:endColNo]
        cls = [str(x) for x in a1.values]
        curnt =None
        for i,x in enumerate(cls):
            curnt = x
            if i >0:
                for j,y in enumerate(cls):
                    if j >0 and y==x and i!=j:
                        cls[j] = y+"_"+str(j)
        return cls
    
    def rename_col(self,df, colNameArray):
        colNameArray = [x.replace(' ', '_') for x in colNameArray if x is not None and isinstance(x,str)]
        xlx1 = df.set_axis(colNameArray, axis=1, inplace=False)
        return xlx1
    
    def get_schema_cols(self,df,colNameArray):
        xlx2 = df[colNameArray]
        return xlx2


class SchemaPrep:
    
    df = None
    cols = None
    schemaNameCol = None
    defaultMaxLength = 0
    
    def __init__(self, df=None, cols=None,schemaNameCol=None):
        self.df = df
        self.cols = cols
        self.schemaNameCol = schemaNameCol       
        
    def get_unq_schema_names(self,df=None,colNm=None):
        df = df if df is not None else self.df        
        colNm = colNm if colNm is not None else self.schemaNameCol
        rtn = None
        if df is not None and colNm is not None:
            a = df[colNm]
            rtn= [str(x) for x in a.values]            
        return rtn
    
    def create_dot_dicts(self,dct,colAr):
        for x in colAr:
            print()
            
        
    def create_dicts(self,pdct,colAr):        
        for x in colAr:
            if "." not in x:
                pdct[x] = {}
        for key in pdct:
            subs = [y[len(key+"."):] for y in colAr if key+"." in y]
            if len(subs)>0:
                self.create_dicts(pdct[key],subs)
                
    def get_table_objs(self,colAr):  
        dct ={}
        self.create_dicts(dct,colAr)
        return dct
    
    def fix_string(inp):
        if inp is not None:
            inp1 = ''
            if isinstance(inp,str):
                dscp = ''.join(dscp.splitlines())
                dscp = re.sub(' +', ' ',dscp)
                return dscp
            else:
                inp1 = str(inp)
    def json_property_template(self,descrp=None,typ=None,key=None, minLength = None , maxLength = None, decim=0):
        dscp = re.sub(' +', ' ',"" if descrp is None else str(descrp).title())
        dscp = ''.join(dscp.splitlines())
        cde = key if descrp is None else str(descrp).title()
        cde = ''.join(cde.splitlines())
        dct = {
            "description":dscp,
            "type":"string" if typ is None else str(typ).lower(),
            "code": cde
        }
        if descrp is None:
            dct["description"] = " "
        else:
            dct["description"] = str(descrp).title()
            
        if typ is None:
            dct["type"] ="string"
        else:
            if isinstance(typ, str):
                if typ.lower() == "object":
                    dct["properties"]={}
                elif typ.lower() == "array":
                    dct["items"]=self.json_property_template(descrp,"object")
                elif typ.lower() == "ref":
                    dct["$ref"]= {}
                elif typ.lower() in ["integer","int","int4"]:
                    dct["type"] = "integer"                     
                elif typ.lower() in [ "char","unit","quan","numc","curr","lang"]:
                    dct["type"] = "string"
                elif typ.lower() in [ "dec","num"]:
                    dct["type"] = "number"
                elif typ.lower() == "double":
                    dct["type"] = "number"
                    dct["format"]="double"                                     
            elif isinstance(typ,float):
                dct["type"] = "number"
                dct["format"]="float" 
                
        if dct["type"] == "string":
            if minLength is not None:
                dct["minLength"]= minLength
            else:
                dct["minLength"]= 0
                
            if maxLength is not None:
                dct["maxLength"]= maxLength
            else:
                dct["maxLength"]= self.defaultMaxLength
        if dct["type"] == "integer" or dct["type"] == "number":
            if decim > 0:
                dct["type"] == "number"
                dct["format"]="double"
                
            if minLength is not None:
                dct["minimum"]= minLength
            else:
                dct["minimum"]= 0
            
            if maxLength is not None:
                dct["maximum"]= self.createMax(maxLength)
            else:
                dct["maximum"]= self.createMax(self.defaultMaxLength)
        return dct 
    
    def createMax(self,mxn=0):
        num = 0
        if mxn >0:            
            n = '9'
            for i in range(0,mxn):
                n = n + '9'
                num =  n 
        if int(num) > 2147483647 :
            return 2147483647
        else:
            return int(num)
            
    def json_schema_template(self,loc=None,title=None,reg=None, adprop=None,descrp=None):
        dct = {
            "$id": None,
            "$schema": "https://json-schema.org/draft/2020-12/schema",
            "title":None,
            "description":None,
            "type":"object",
            "propertyNames":{
                "pattern":None
            },
            "properties":{
                
            },
            "additionalProperties":False if adprop is None else adprop
            
        }
        dct["$id"] = "".join(loc if loc is not None else "https://localhost/schema/{}")
        ttl = "".join(str(title).splitlines() if title is not None else " Json Scheam design for {}")
        ttl = ''.join(ttl.splitlines())
        dct["title"]=re.sub(' +', ' ',ttl) 
        dscp = "".join(descrp.splitlines() if descrp is not None else " This is Description {}")
        dscp = ''.join(dscp.splitlines())
        dct["description"]=re.sub(' +', ' ',dscp)
        dct["propertyNames"]["pattern"]= "".join("^[A-Za-z0-9_]+$" if reg is None else reg)
        if adprop is None or adprop == True:
            dct["additionalProperties"]="true"
        else:
            dct["additionalProperties"]="false"
        return dct
        

class GenerateSchema:
    scma = None
    sp = None
    xlx = None
    scDf = None    
    inps = None
    defaultMaxLength = 255
    def __init__(self,dct=None):
        
        self.inps = dct
        
        self.le = LoadExcel(dct['filePath'],dct['fileName'],dct['columnNames'],
                            dct['sheetNames'],dct['columnNameRowIndex'])
        self.sp = SchemaPrep()
        #print(dct['maxLength'],'-- man keb')
        if dct['maxLength'] is not None and dct['maxLength'] > 0:
            self.sp.defaultMaxLength = dct['maxLength'] 
        else:
            self.sp.defaultMaxLength = defaultMaxLength
    
    def loadFle(self,sheetName=None,rename=None,rowno=None, startColNo=None,endColNo=None):
        sheetName = sheetName if sheetName is not None else self.inps['sheetName']
        rename = rename if rename is not None else self.inps['rename']   
        
        xlx = self.le.loadFile(sheetName if sheetName is not None else "Sheet1") 
        #print(xlx.columns)
        if rename is True:            
            rowno = rowno if rowno is not None else self.inps['rowno'] 
            startColNo = startColNo if startColNo is not None else self.inps['startColNo']
            endColNo = endColNo if endColNo is not None else self.inps['endColNo']
            
            clns = self.le.get_col_name_frm_row(xlx,rowno,startColNo,endColNo)
            xlx2 = self.le.rename_col(xlx, clns)
            #print("--------")
            #print(xlx2.columns)
            self.xlx = xlx2
            #print("--------")
            #print(self.xlx.columns)
            self.xlx.head(5)
        else:
            self.xlx = xlx
        return self.xlx
    
    def optaion_schema_frame(self, schemaCols=None):
        # create sub Data Frame for selecting columns which have schema prep columns
        schemaCols = schemaCols if schemaCols is not None else self.inps['schemaCols']
        #print(schemaCols)
        #print(self.xlx.columns)
        j1 = self.le.get_schema_cols(self.xlx,schemaCols)
        self.sp.df = j1
        self.scDf = j1
    
    def generate_table_name_object(self,schemaNmsCol=None):
        schemaNmsCol  = schemaNmsCol if schemaNmsCol is not None else self.inps['schemaNmsCol']       
        aa = self.sp.get_unq_schema_names(self.sp.df,schemaNmsCol)[1:]
        dct =self.sp.get_table_objs(aa)
        return dct
    
    def generate_table(self):
        loc="https://localhost/schema/{}.schema.json",
        title="Final NoSQL JSON Schema",
        reg="^[A-Za-z0-9_]+$", 
        adprop=False,
        descrp=re.sub(' +', ' ', "This is Final Master Attribute Schema \
                which is hosted for Keeping Data From NoSQL Schema")
        jsn = self.sp.json_schema_template(loc,title,reg,adprop,descrp)
        return jsn
        
    def generate_table_prperties(self, tables,jsn,mtchToken=None):
        d1 =None
        currentItrKey = None
        for key, value in tables.items():           
            jsn["properties"][key] = self.sp.json_property_template(key,"object",key)
            d1 = self.scDf[self.scDf.Schema_8 == key]
           
            for row in d1.itertuples():
                #print(row.Field)  print(row.Length) print(row.Type) print(row.Comment)
                entryType = row.Data_Type_10
                if row.CustomType is not None and isinstance(row.CustomType,str) == True and row.CustomType.lower() == 'array':
                    entryType = 'array'
                   
                jsn["properties"][key]["properties"][row.Field] = self.sp.json_property_template(row.Comment,entryType, row.Comment,0,row.Length_11,row.Decimal_12)
                
            if mtchToken is not None:
                d2 = self.scDf[self.scDf.Schema_8 == mtchToken+"."+key]
                for row in d2.itertuples():
                    entryType = row.Data_Type_10
                    if row.CustomType is not None and isinstance(row.CustomType,str) == True and row.CustomType.lower() == 'array':
                        entryType = 'array'
                    jsn["properties"][key]["properties"][row.Field] = self.sp.json_property_template(row.Comment,entryType,row.Comment,0,row.Length_11,row.Decimal_12)
            #print(value)           
            #print("------------>>")
            if len(value)>0:
                #print("Has child tables")
                mtchToken1 = key
                if mtchToken is not None:
                    mtchToken1 = mtchToken+"."+key               
                for k, v in value.items():
                    #print("Generating Schema --- Start ---mtchToken :: "+ mtchToken1)
                    self.generate_table_prperties(value,jsn["properties"][key],mtchToken1)       
        #print("Generating Schema --- End ---")
        return jsn
    
    def runfx(self, filePath = None,fileName = None):
        self.loadFle()
        #print(self.xlx.size)
        self.xlx.head(2)
        self.optaion_schema_frame()
        tables = self.generate_table_name_object()  
        jsn = self.generate_table()
        self.generate_table_prperties(tables,jsn)
        if filePath is not None and fileName is not None:
            with open(filePath+"/"+fileName+".json",'w') as f:
                json.dump(jsn,f,indent=2)
        else:
            print()        
        return jsn


class UpdateRelation:
    inps = None
    xlx = None
    jsond = None
    def __init__(self,dct=None):
        self.inps = dct
        self.jsond = dct['jsond']
        self.le = LoadExcel(dct['filePath'],dct['fileName'],dct['columnNames'],
                            dct['sheetNames'],dct['columnNameRowIndex'])
    
    def loadFle(self,sheetName=None,rename=None,rowno=None, startColNo=None,endColNo=None):
        sheetName = sheetName if sheetName is not None else self.inps['sheetName']
        rename = rename if rename is not None else self.inps['rename']   
        
        xlx = self.le.loadFile(sheetName if sheetName is not None else "Sheet3") 
        #print(xlx.columns)
        if rename is True:            
            rowno = rowno if rowno is not None else self.inps['rowno'] 
            startColNo = startColNo if startColNo is not None else self.inps['startColNo']
            endColNo = endColNo if endColNo is not None else self.inps['endColNo']
            
            clns = self.le.get_col_name_frm_row(xlx,rowno,startColNo,endColNo)
            xlx2 = self.le.rename_col(xlx, clns)
            #print("--------")
            #print(xlx2.columns)
            self.xlx = xlx2
            #print("--------")
            #print(self.xlx.columns)
            self.xlx = self.xlx.sort_values(by=['Schema'], ascending=True, na_position='first')
            self.xlx = self.xlx.dropna()
            
        else:
            self.xlx = xlx
            self.xlx = self.xlx.sort_values(by=['Schema'], ascending=True, na_position='first')
            self.xlx = self.xlx.dropna()
        return self.xlx
    
    def getObjectTansforms(self,df=None):
        obj_arr = []
        if df is None:
            df = self.xlx
        for row in df.itertuples():
            objs = row.Schema
            obj_arr.append([row.Relation,objs.split(".")])
            
        return obj_arr
    
    def find_and_alter_nested_item(self, obj, key, nested_item = None,typ=None, cntr=None,end=None):
        if key in obj:
            if cntr > 0 and end > 0 and cntr == end:
                self.alter_p(obj,key,typ);
            nested_item = obj[key]
        for k, v in obj.items():
            if isinstance(v,dict):
                item = self.find_and_alter_nested_item(v, key, nested_item,typ,cntr,end)
                if item is not None:
                    nested_item = item
        return nested_item
    
    def alter_p(self,obj,key,typ):
        if typ.lower() == 'array':
            tempo = obj[key].copy()
            obj[key] = {'description': tempo['description'],
                                            'type': typ.lower(), 
                                            'code': tempo['code'],
                                            'items':{
                                                'type':'object',
                                                'properties':tempo['properties']
                                            }
                                           }
        elif typ.lower() == 'object':
            if obj[key]['type'] != 'object':
                tempo = obj[key].copy()
                obj[key] = {'description': tempo['description'],
                                            'type': typ.lower(), 
                                            'code': tempo['code'],
                                            'properties':tempo
                                           }
    def alterObject(self, jsond, tempjson, entry):
        if jsond is not None and entry is not None :
            typ = entry[0]
            keys = entry[1]
            
            if tempjson is None:
                tempjson = jsond
            if len(keys) == 1:
                if 'properties'  in jsond:
                    if keys[0] in jsond['properties']:
                        tmpo = jsond['properties'][keys[0]].copy()
                        #print(keys[0])
                        #print('------1-------')
                        #print(tmpo)
                        #print('------1--------')
                        if typ.lower() == 'array':
                            jsond['properties'][keys[0]] = {'description': tmpo['description'],
                                                            'type': typ.lower(), 
                                                            'code': tmpo['code'],
                                                            'items':{
                                                                'type':'object',
                                                                'properties':tmpo['properties']
                                                            }
                                                           }
                        elif typ.lower() == 'object':
                            if jsond['properties'][keys[0]]['type'] != 'object':
                                jsond['properties'][keys[0]] = {'description': tmpo['description'],
                                                            'type': typ.lower(), 
                                                            'code': tmpo['code'],
                                                            'properties':tmpo
                                                           }
            else:
                nest_item = jsond
                cntr = 0 
                end = len(keys)
                for key in keys:
                    cntr = cntr + 1
                    deepest_item = self.find_and_alter_nested_item(nest_item, key, nest_item,typ, cntr,end)
                cntr = 0
                #print('deepest_item for -------', keys)
                #print(deepest_item)
        return jsond
    
    def alterJson(self, jsond, tempjson,arr):
        if arr is not None and json is not None:
            for entry in arr:
                #print(entry)
                if isinstance(entry[0],str) == True and len(entry[1])>0:                   
                    self.alterObject(jsond,tempjson, entry) 
                        
                        
    
#----------------------------------------------------------    
# Class utilized for Display At Notebook
'''
    Class utilized for Display At Notebook
    https://stackoverflow.com/questions/18873066/pretty-json-formatting-in-ipython-notebook
    https://github.com/caldwell/renderjson
'''
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 700px; width:100%;" class="renderjson"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_icons("+", "-");
          renderjson.set_show_to_level(1);
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [50]:
pth = '/Users/anand/Documents/sayansi/workspaces/condaws/xlsx-json'
inpss = {
        "filePath" : pth+"/samplefile",
        "fileName" : "kfk.xlsx",
        "columnNames" : ["Source Topic",
            "Schema","Attribute Name","Data Type","Length",
            "Decimal","SM",
            "Topc","Schema","Field","Data Type","Length","Decimal","Comment","CustomType"],
        "columnNameRowIndex" : 0,
        "schemaCols" : ["Schema_8", "Field", "Data_Type_10", "Length_11","Decimal_12", "Comment","CustomType"],
        "sheetNames":["Sheet1","Sheet2"],
        "rename":True,
        "rowno":0, 
        "startColNo":0,
        "endColNo":15,
        "schemaNmsCol":"Schema_8",
        "sheetName":"Sheet1",
        "maxLength":50
    }

gs = GenerateSchema(inpss)
#tables = gs.runfx()
tables = gs.runfx(pth+'/samplefile','result')
#print(json.dumps(tables,indent=2))

urinps = {
        "filePath" : pth+"/samplefile",
        "fileName" : "kfk.xlsx",
        "columnNames" : ["Segment","Schema","Relation"],
        "columnNameRowIndex" : 0,
        "schemaCols" : ["Segment","Schema","Relation"],
        "sheetNames":["Sheet1","Sheet2","Sheet2"],
        "rename":True,
        "rowno":0, 
        "startColNo":0,
        "endColNo":15,
        "schemaNmsCol":"Schema",
        "sheetName":"Sheet3",
        "maxLength":50,
        "jsond": tables
    }
ur = UpdateRelation(urinps);
ur.loadFle()
#ur.xlx = ur.xlx.sort_values(by=['Schema'], ascending=True, na_position='first')

dp = ur.getObjectTansforms()
ur.alterJson(ur.jsond,None,dp)
RenderJSON(ur.jsond)



In [49]:
class CreateSampleJons:
    def __init__(self,jso):
        print(jso)

def create_sampleJson(df,jso):
    print(df.size)
    if jso is None:
        jso = {}
    for row in df.itertuples():
        #print(row.Segment,row.Schema,row.Relation)
        isObj = True;
        if row.Relation.lower() =='object':
            jso[str(row.Segment)] = {}
        elif row.Relation.lower() =='array':
            jso[str(row.Segment)] = []
            isObj = False
            
        props = row.Schema.split(".")
        ob = {}
        #print('--------------------')
        if str(row.Segment) in jso.keys():
            for prop in props:
                if isObj == True:
                    jso[row.Segment][prop]={}
                else:
                     ob[prop]=None
            if isObj == False:
                jso[row.Segment].append(ob)
    return jso
        
    print(jso)    

#smp = create_sampleJson(ur.xlx,None)
#ur.xlx.head(20)
#RenderJSON(smp)